In [2]:
import numpy as np
from os import listdir
from pickle import dump
from pickle import load
import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint


import matplotlib.pyplot as plt
import pickle as pkl
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


## 1 图片预处理

In [3]:
# import cv2

# '''----------------test-------------------'''
# test = cv2.imread( 'Flickr8k_Dataset\\Flicker8k_Dataset\\'+img_names[0])
# print('h,w,c = ',test.shape)
# test = cv2.resize(test,(224,224))
# print('h,w,c = ',test.shape)

In [4]:
import cv2
def reshape(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224))  
    return img

In [5]:
import os
#os.listdir(filename)返回filename中所有文件的文件名列表
img_names = os.listdir('Flickr8k_Dataset\Flicker8k_Dataset')
img_num = len(img_names)
img_num

8091

In [6]:
# '''----------------test-------------------'''
# x = np.expand_dims(test, axis=0)  # 展开
# x = preprocess_input(x)       # 预处理到0～1
# out = vgg19.predict(x)

# print(out.shape)
# type(out)

In [7]:
img_mapping = dict()  
for i in range(img_num):
    img_path =  'Flickr8k_Dataset\\Flicker8k_Dataset\\'+img_names[i]
    img = reshape(img_path)
    name = img_names[i]
    img_mapping[name] = img

In [8]:
'''===============test==============='''
img_mapping[img_names[1]].shape

(224, 224, 3)

In [9]:
import pickle

if not os.path.exists(os.path.abspath('.') + '\\img_mapping.npy'):    
    with open('img_mapping.pkl', 'wb') as df1:
        pickle.dump(img_mapping, df1, pickle.HIGHEST_PROTOCOL)
else:
    with open('img_mapping.pkl', 'rb') as df2:
        img_mapping = pickle.load(df2)

## 2 处理语句信息

In [10]:
import string

# load .txt file as text
def load_doc(txt_filepath):
    # open the file as read only
    file = open(txt_filepath, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [11]:
# extract descriptions for images
def load_text(text):
    imgs=[]
    words = []
    img_ids = []
    annots = []
    for line in text.split('\n'):
        
        tokens = line.split()
        if len(line) > 1:
            
            # take the first token as the image id, the rest as the description
            img_id, img_wd = tokens[0], tokens[1:]
            # remove filename from image id
            img_id = img_id.split('#')[0]
            
            #build the list of ids
            if img_id not in img_ids:
                img_ids.append(img_id)
                imgs.append(img_mapping[img_id])
                annots.append([])
            
            # build a set of words
            for word in img_wd:
                word.strip('.,\"')
                word.lower()
                if word not in words:
                    words.append(word)
                    
            # convert description tokens back to string        
            annot = ' '.join(img_wd)
            #build the list of ids
            annots[-1].append(annot)
            
    return img_ids,imgs, annots, words


In [12]:
text_path = 'Flickr8k_text\Flickr8k.lemma.token.txt'
text = load_doc(text_path)
ids, imgs, annots, words =load_text(text)

In [13]:
import random
from random import shuffle
#import itertools 
z = list(zip( imgs, annots))
random.shuffle(z)
imgs, annots = zip(*z)

In [14]:
wd_len = len(words)
wd_len

7405

In [15]:
#annots = np.array(annots)
# annots.shape

In [16]:
[item for sublist in annots for item in sublist]

['A dirt biker be take a large jump wear a yellow suit .',
 'A man in yellow performs a bike trick .',
 'A motocross bike be jump through the air over a dirt track .',
 'A person wear yellow be take a jump on a dirt bike .',
 'A motorcycle racer in a yellow uniform be in midair .',
 'A man in a white tank hit a jump with his red Bmx bicycle .',
 'a man be make a red and black bicycle jump in a city skate park .',
 'A man wear a white tank top be ride a red bike along a jump .',
 'A young man jump a bicycle off a sloped wall over a bench .',
 'Cyclist ride on a cement slope .',
 'a mountain biker be ride on a mark out competition trail in the wood .',
 'A person wear red ride a bicycle over a dirt path .',
 'A woman ride a bike on a dirt path .',
 'Bicyclist ride on a dirt race course .',
 'Mountain bike rider on a dirt trail .',
 'A boy turn a handcrank in a courtyard .',
 'A child turn a crank with bench behind him .',
 'A young boy be stand on a gray tiled area .',
 'A young boy stan

In [17]:
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts([item for sublist in annots for item in sublist])
print(tokenizer.word_index) 

{'a': 1, 'in': 2, 'be': 3, 'on': 4, 'the': 5, 'dog': 6, 'and': 7, 'man': 8, 'with': 9, 'of': 10, 'two': 11, 'boy': 12, 'girl': 13, 'woman': 14, 'person': 15, 'white': 16, 'black': 17, 'run': 18, 'play': 19, 'wear': 20, 'to': 21, 'stand': 22, 'at': 23, 'jump': 24, 'water': 25, 'child': 26, 'red': 27, 'young': 28, 'an': 29, 'brown': 30, 'his': 31, 'sit': 32, 'blue': 33, 'shirt': 34, 'through': 35, 'walk': 36, 'while': 37, 'hold': 38, 'ball': 39, 'down': 40, 'little': 41, 'ride': 42, 'grass': 43, 'snow': 44, 'look': 45, 'over': 46, 'three': 47, 'front': 48, 'small': 49, 'field': 50, 'up': 51, 'large': 52, 'by': 53, 'green': 54, 'group': 55, 'one': 56, 'yellow': 57, 'her': 58, 'into': 59, 'air': 60, 'beach': 61, 'mouth': 62, 'near': 63, 'rock': 64, 'player': 65, 'dress': 66, 'bike': 67, 'street': 68, 'another': 69, 'for': 70, 'its': 71, 'from': 72, 'as': 73, 'climb': 74, 'watch': 75, 'outside': 76, 'other': 77, 'hat': 78, 'off': 79, 'smile': 80, 'out': 81, 'mountain': 82, 'next': 83, 'jack

In [18]:
annots_=annots
annots_
k = 0
while k < 8091:
    p = 0
    while p < 5:
        a = annots_[k][p]
        a = T.text_to_word_sequence(a)
        a = tokenizer.texts_to_sequences(a)
        a = [item for sublist in a for item in sublist]
        #a = np.array(a)
        print(a)
        annots_[k][p]=a
        p += 1
    k += 1
#annots_ = np.array(annots_)
annots_

[1, 108, 250, 3, 121, 1, 52, 24, 20, 1, 57, 181]
[1, 8, 2, 57, 1807, 1, 67, 122]
[1, 639, 67, 3, 24, 35, 5, 60, 46, 1, 108, 174]
[1, 15, 20, 57, 3, 121, 1, 24, 4, 1, 108, 67]
[1, 196, 538, 2, 1, 57, 155, 3, 2, 241]
[1, 8, 2, 1, 16, 491, 299, 1, 24, 9, 31, 27, 492, 127]
[1, 8, 3, 163, 1, 27, 7, 17, 127, 24, 2, 1, 175, 256, 102]
[1, 8, 20, 1, 16, 491, 118, 3, 42, 1, 27, 67, 114, 1, 24]
[1, 28, 8, 24, 1, 127, 79, 1, 2643, 112, 46, 1, 154]
[417, 42, 4, 1, 497, 456]
[1, 82, 250, 3, 42, 4, 1, 935, 81, 610, 294, 2, 5, 194]
[1, 15, 20, 27, 42, 1, 127, 46, 1, 108, 171]
[1, 14, 42, 1, 67, 4, 1, 108, 171]
[445, 42, 4, 1, 108, 93, 335]
[82, 67, 230, 4, 1, 108, 294]
[1, 12, 333, 1, 3892, 2, 1, 878]
[1, 26, 333, 1, 2644, 9, 154, 98, 142]
[1, 28, 12, 3, 22, 4, 1, 141, 1941, 146]
[1, 28, 12, 22, 2, 1, 878, 333, 1, 2644]
[28, 12, 333, 1, 2644, 2, 1, 68]
[5, 12, 200, 103, 40, 4, 1, 105, 3, 3, 326, 114, 5, 159, 53, 69, 12]
[11, 13, 19, 4, 1, 105, 2, 1, 878]
[11, 15, 19, 4, 1, 186, 105]
[11, 49, 26, 2, 27

[1, 12, 24, 46, 101, 25]
[1, 12, 20, 96, 366, 24, 46, 101, 504, 25, 2, 1, 531]
[1, 41, 12, 2, 248, 24, 59, 1, 432, 10, 25]
[1, 28, 12, 24, 35, 1, 2134, 10, 25]
[28, 12, 9, 248, 19, 2, 1, 504]
[1, 17, 7, 16, 6, 22, 4, 5, 43, 9, 1, 187, 2, 31, 62]
[1, 489, 10, 1, 49, 17, 7, 16, 6, 9, 1, 187, 2, 71, 62]
[17, 7, 16, 6, 38, 1, 187, 2, 62]
[30, 7, 16, 6, 9, 187, 2, 62, 50, 7, 110]
[30, 7, 16, 6, 9, 1, 187, 2, 71, 62]
[1, 14, 45, 23, 1, 457, 2, 1, 1821, 37, 38, 1, 135]
[1, 14, 20, 1, 17, 34, 1961, 35, 1, 457, 23, 1, 370]
[1, 14, 9, 1, 135, 1961, 2, 1, 1821]
[1, 14, 9, 1, 135, 45, 35, 1, 457, 581]
[14, 38, 1, 135, 312, 70, 457]
[1, 153, 202, 20, 1, 151, 84, 2668, 1, 3946, 1453, 23, 1, 452]
[1, 153, 14, 3, 223, 2, 1, 406, 452]
[1, 898, 14, 223, 274]
[1, 14, 223, 1, 52, 274, 2, 1, 452]
[1, 14, 3, 223, 274, 23, 1, 452]
[1, 49, 16, 6, 3, 2, 1, 683, 24, 2, 48, 10, 1, 811]
[1, 16, 6, 180, 21, 74, 4, 1, 2371]
[1, 16, 6, 180, 21, 24, 4, 1, 811]
[1, 16, 6, 9, 85, 797, 3, 24, 51, 21, 536, 1, 2371]
[16, 

[1, 14, 22, 83, 21, 1, 8, 232, 3, 32, 4, 1, 64]
[1, 55, 10, 15, 3, 1126, 23, 1, 61]
[1, 55, 10, 3139, 348, 63, 101, 349, 23, 1, 61]
[131, 387, 4, 1, 61, 83, 21, 176]
[178, 2686, 3, 2380, 63, 392, 40, 176]
[178, 15, 19, 7, 866, 4, 1, 61]
[11, 17, 7, 16, 6, 4, 1, 386, 61, 236, 46, 1, 186, 106]
[11, 6, 109, 5, 578, 106, 2, 89, 62, 73, 364, 36, 4, 5, 129]
[11, 6, 38, 56, 106, 4, 1, 61]
[11, 6, 4, 1, 61, 236, 46, 1, 106]
[11, 6, 4, 271, 129, 134, 302, 5, 578, 106]
[1, 12, 187, 216, 20, 89, 155]
[1, 49, 12, 80, 73, 243, 22, 9, 280, 20, 5, 578, 17, 78]
[26, 412, 89, 1710, 1391, 237]
[12, 3, 328, 20, 5, 578, 187, 78]
[47, 12, 20, 1710, 3966, 78]
[1, 17, 6, 164, 69, 17, 6, 40, 1, 43, 160, 125]
[56, 17, 6, 7, 56, 2687, 577, 6, 18, 40, 1, 125]
[11, 17, 6, 18, 35, 1, 50]
[11, 6, 93, 126, 1, 50, 4, 1, 546, 330]
[11, 6, 18, 2, 1, 50, 10, 465, 43, 4, 1, 554, 330]
[1, 17, 6, 601, 35, 5, 44]
[1, 17, 6, 601, 2, 5, 44]
[1, 52, 17, 6, 3, 601, 2, 5, 443, 44]
[1, 49, 17, 7, 30, 6, 3, 22, 2, 5, 44]
[17, 6, 6

[1, 8, 334, 31, 105, 2, 241, 221, 5, 368, 159]
[1, 170, 87, 1, 24, 63, 101, 1827, 110]
[321, 2389, 170, 177, 188, 87, 1, 122, 21, 3989, 31, 618]
[1, 6, 421, 4, 31, 138, 2, 5, 43]
[1, 52, 151, 6, 3, 421, 97, 4, 71, 138, 2, 1, 255, 10, 43]
[1, 52, 151, 6, 3, 421, 2, 54, 43]
[191, 57, 6, 421, 97, 4, 1, 54, 374]
[52, 6, 421, 4, 71, 138, 2, 54, 43]
[17, 626, 109, 27, 464, 99, 37, 19, 9, 1, 30, 626]
[17, 626, 9, 99, 2, 62]
[56, 17, 6, 9, 1, 99, 7, 56, 57, 6]
[858, 11, 626, 56, 17, 7, 56, 30, 3, 19]
[11, 626, 19, 9, 1, 27, 99]
[1, 8, 36, 1, 1454, 529, 21, 5, 159, 356, 11, 91]
[1, 8, 36, 126, 1, 1454]
[1, 8, 36, 4, 1, 529, 1454, 214, 1, 91]
[1, 28, 8, 3, 36, 126, 1, 894, 190, 23, 297]
[1, 28, 8, 36, 1, 1454, 530, 21, 1, 91, 23, 297]
[1, 6, 109, 1, 303, 2, 71, 62]
[1, 550, 6, 109, 1, 17, 303, 2, 71, 62]
[1, 52, 575, 30, 6, 3, 36, 9, 1, 303, 2, 31, 62]
[1, 57, 6, 109, 1, 17, 303]
[6, 36, 9, 31, 2152, 2, 31, 62]
[1, 6, 325, 93, 35, 5, 44]
[1, 6, 325, 216, 7, 89, 818]
[1, 2696, 117, 7, 80, 98, 31,

[1, 16, 6, 100, 1, 27, 166, 2, 5, 43]
[1, 16, 6, 24, 2, 5, 60, 21, 100, 1, 27, 166]
[1, 6, 156, 21, 100, 5, 166, 2, 71, 62]
[1, 13, 2, 86, 1534, 1, 797]
[1, 41, 13, 19, 9, 1, 797]
[41, 13, 22, 2, 48, 10, 273, 112, 4, 497, 451]
[1, 13, 3, 20, 1, 16, 34, 9, 86, 401, 7, 433, 3, 22, 2, 48, 10, 1, 313]
[1, 49, 13, 2, 1300, 3, 20, 27, 1535, 1, 27, 401, 7, 1, 16, 118]
[1, 12, 3, 4, 1, 127, 7, 182, 3, 1, 13, 4, 5, 138, 10, 144, 9, 29, 418]
[1, 208, 22, 76, 9, 29, 418, 7, 1, 127]
[1, 13, 9, 29, 418, 3, 42, 4, 5, 138, 10, 1, 127, 3114, 53, 1, 80, 12]
[1, 8, 42, 1, 67, 9, 1, 14, 38, 29, 418]
[182, 3, 1, 12, 4, 1, 127, 7, 1, 13, 38, 29, 418, 74, 4, 5, 138]
[1, 172, 156, 46, 1, 161, 37, 1, 230, 205, 4]
[8, 42, 1, 172, 24, 46, 1, 195, 161]
[15, 2, 1, 57, 150, 3, 24, 1, 30, 172, 46, 1, 161]
[5, 172, 7, 230, 3, 24, 46, 1, 161]
[5, 172, 7, 230, 24, 1, 161]
[1, 26, 986, 31, 133, 81, 72, 101, 229]
[1, 26, 9, 33, 354, 32, 1736, 229]
[1, 115, 19, 2, 1, 595, 10, 229]
[1, 30, 268, 270, 3, 38, 240, 5, 279, 10

...

[1, 12, 19, 9, 1, 120]
[12, 19, 9, 1, 420]
[1, 135, 2, 2303, 32, 53, 1, 99]
[1, 41, 12, 9, 191, 30, 354, 3, 540, 1, 41, 120]
[1, 270, 2, 2303, 19, 9, 1, 420]
[1, 8, 38, 1, 272, 83, 21, 1, 6345, 1768]
[1, 8, 9, 1, 6346, 272, 22, 2, 5, 44, 53, 383]
[1, 15, 246, 51, 1, 383, 211, 1, 594, 77, 383, 4, 5, 137, 159]
[82, 310, 3, 246, 51, 89, 383]
[9, 1, 676, 7, 189, 239, 4, 5, 159, 1, 8, 20, 1, 376, 184, 3, 38, 1, 383, 37, 77, 383, 3, 6347, 246, 51]
[1, 30, 1819, 1874, 4, 1, 13, 143, 104]
[1, 13, 2, 1, 219, 78, 92, 1, 1306, 2036, 436, 4, 58, 104]
[1, 13, 2, 1, 78, 92, 101, 1370, 10, 2061, 4, 58, 535]
[1, 13, 2, 1, 530, 1884, 78, 92, 1, 1819, 4, 58, 104]
[1, 13, 2, 1, 57, 33, 7, 54, 530, 1884, 78, 92, 1, 1306, 45, 6348, 4, 58, 104]
[1, 12, 183, 40, 1, 116]
[1, 12, 20, 1, 283, 34, 9, 29, 414, 272, 183, 40, 1, 116, 9, 31, 152, 1596, 81]
[1, 80, 12, 183, 40, 1, 30, 116]
[12, 116, 40, 1, 116, 9, 757, 152]
[1, 115, 3, 116, 40, 1, 151, 342, 116]
[1, 55, 10, 26, 19, 111, 7, 212, 76, 1, 452]
[1, 55, 10

[6380, 610]
[29, 966, 239, 6381, 221, 1, 311, 37, 588, 75]
[1, 113, 10, 15, 20, 27, 34, 18, 114, 1, 175, 147]
[1, 55, 10, 15, 2, 27, 3, 18, 40, 1, 147]
[1, 52, 55, 10, 8, 7, 14, 328, 20, 27, 283, 34, 836, 2, 1, 93]
[131, 2, 85, 283, 34, 3, 677, 4, 1, 68]
[182, 3, 2363, 10, 15, 9, 27, 34, 18, 1, 1578]
[1, 13, 2, 1, 16, 34, 3, 32, 4, 1, 102, 154, 9, 1, 6, 83, 21, 58]
[1, 14, 223, 4, 1, 154, 37, 1, 30, 7, 16, 303, 6, 22, 83, 21, 58]
[1, 14, 3, 32, 4, 1, 154, 9, 1, 3403, 2, 58, 681, 7, 1, 16, 6, 4, 1, 33, 303, 21, 58, 168]
[1, 14, 32, 4, 1, 154, 23, 1, 102, 9, 58, 6, 2, 48, 10, 58]
[14, 32, 4, 154, 7, 38, 5, 303, 10, 1, 52, 16, 7, 30, 6]
[1, 55, 10, 28, 153, 26, 56, 109, 1, 1783, 94, 2, 48, 10, 1, 313]
[290, 26, 3, 22, 2, 48, 10, 1, 195, 1203]
[290, 115, 94, 70, 1, 123, 56, 3, 38, 1, 642]
[119, 1205, 7, 56, 1560, 94, 70, 1, 123, 80]
[182, 3, 290, 115, 22, 83, 21, 134, 77, 7, 5, 56, 2, 27, 92, 1, 642]
[1, 741, 2, 1, 27, 155, 94, 2, 1, 523]
[1, 1012, 94, 70, 5, 88, 2, 1, 27, 7, 16, 66]
[1, 1

[29, 6424, 3095, 1060, 2, 1169, 1634, 2, 91, 221]
[1, 14, 205, 72, 1, 91, 58, 264, 876, 2, 821]
[1, 14, 1489, 72, 1, 33, 1169, 2195, 21, 1, 91]
[14, 205, 4, 91, 1327]
[1, 8, 2, 1, 376, 84, 7, 893, 237, 22, 9, 31, 152, 51, 23, 31, 168, 2, 1, 44, 684]
[1, 8, 3, 22, 4, 108, 9, 31, 104, 2, 5, 60, 9, 82, 7, 44, 98, 142]
[1, 8, 444, 31, 152, 4, 215, 61]
[56, 474, 8, 9, 1704, 2, 5, 60, 9, 52, 215, 82, 2, 5, 107]
[6425, 9, 757, 152, 22, 4, 44, 475, 64]
[1, 8, 2, 201, 132, 3, 1498, 4, 1, 554, 330]
[1, 8, 3, 42, 1, 27, 1493]
[1, 8, 2216, 2, 5, 25]
[1, 1493, 3, 163, 1, 191, 199, 2, 5, 130, 309, 42, 1, 27, 1498]
[1, 1493, 226, 51, 432]
[1, 13, 3, 20, 1, 52, 33, 7, 27, 78]
[1, 13, 20, 1, 2131, 272, 1014, 78, 32, 4, 5, 368]
[1, 14, 32, 9, 20, 1, 1985, 1136, 3032, 78]
[1, 14, 9, 1, 314, 27, 16, 7, 33, 78, 32, 4, 5, 159]
[13, 9, 1, 52, 78, 9, 1, 2131, 272, 4, 144]
[1, 689, 451, 5, 461, 9, 2434, 72, 1, 1050, 278, 1, 260, 158]
[1, 260, 65, 180, 21, 163, 1, 461, 37, 5, 689, 7, 69, 65, 45, 4]
[1, 260, 19,

[1, 28, 13, 2, 1, 301, 57, 66, 245, 53, 57, 229]
[1, 28, 13, 20, 57, 200, 4, 5, 159]
[1, 13, 9, 1, 57, 66, 7, 962, 4, 1076, 245, 53, 57, 229]
[1, 49, 13, 2, 1, 57, 66, 200, 4, 1, 2450, 245, 53, 229]
[1, 8, 3, 304, 7, 24, 79, 10, 1, 44, 125]
[1, 15, 2, 1, 17, 84, 3, 304, 278, 5, 1599]
[1, 790, 10, 1, 15, 304, 35, 1, 468, 10, 44]
[1, 210, 177, 79, 1, 44, 2037, 9, 1, 219, 295, 2, 5, 107]
[1, 15, 2, 1, 1419, 3, 4, 1, 304]


([[1, 108, 250, 3, 121, 1, 52, 24, 20, 1, 57, 181],
  [1, 8, 2, 57, 1807, 1, 67, 122],
  [1, 639, 67, 3, 24, 35, 5, 60, 46, 1, 108, 174],
  [1, 15, 20, 57, 3, 121, 1, 24, 4, 1, 108, 67],
  [1, 196, 538, 2, 1, 57, 155, 3, 2, 241]],
 [[1, 8, 2, 1, 16, 491, 299, 1, 24, 9, 31, 27, 492, 127],
  [1, 8, 3, 163, 1, 27, 7, 17, 127, 24, 2, 1, 175, 256, 102],
  [1, 8, 20, 1, 16, 491, 118, 3, 42, 1, 27, 67, 114, 1, 24],
  [1, 28, 8, 24, 1, 127, 79, 1, 2643, 112, 46, 1, 154],
  [417, 42, 4, 1, 497, 456]],
 [[1, 82, 250, 3, 42, 4, 1, 935, 81, 610, 294, 2, 5, 194],
  [1, 15, 20, 27, 42, 1, 127, 46, 1, 108, 171],
  [1, 14, 42, 1, 67, 4, 1, 108, 171],
  [445, 42, 4, 1, 108, 93, 335],
  [82, 67, 230, 4, 1, 108, 294]],
 [[1, 12, 333, 1, 3892, 2, 1, 878],
  [1, 26, 333, 1, 2644, 9, 154, 98, 142],
  [1, 28, 12, 3, 22, 4, 1, 141, 1941, 146],
  [1, 28, 12, 22, 2, 1, 878, 333, 1, 2644],
  [28, 12, 333, 1, 2644, 2, 1, 68]],
 [[5, 12, 200, 103, 40, 4, 1, 105, 3, 3, 326, 114, 5, 159, 53, 69, 12],
  [11, 13, 19, 

In [19]:
# annots_.shape

In [20]:
# '''----------------test-------------------'''
# import keras.preprocessing.text as T
# from keras.preprocessing.text import Tokenizer
# tokenizer = Tokenizer(num_words=None)
# tokenizer.fit_on_texts(texts)
# text1='some thing to eat'
# text2='some thing to drink'
# texts=[text1,text2]

# print (T.text_to_word_sequence(text2) )

## 3 制做数据集

In [21]:
from sklearn.model_selection import train_test_split
img_train, img_test, annot_train, annot_test = train_test_split(imgs,annots_,test_size=0.25, random_state=42)
img_test, img_val, annot_test, annot_val = train_test_split(img_test, annot_test, test_size=0.5, random_state=42)

## 4 搭建模型

In [22]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

6447

In [23]:
from keras.utils import plot_model
# define the decoder model
def define_model(vocab_size, max_length):
    
    pretrained_cnn = VGG19(weights='imagenet', include_top=False)
    pretrained_cnn.layers.pop()
    # pretrained_cnn.trainable = True
    for layer in pretrained_cnn.layers[:]:
        layer.trainable = True
    
    # feature extractor model
    inputs1 = Input(shape=(224,224,3))
#     Encoder = VGG19(weights='imagenet', include_top=True)
#     Encoder.layers.pop()
    fe0 = pretrained_cnn(inputs1)
    fe1 = GlobalAveragePooling2D()(fe0)
    fe2 = Dropout(0.5)(fe1)
    fe3 = Dense(256, activation='relu')(fe2)

    # sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    # decoder model
    decoder1 = add([fe3, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    
    return model

In [24]:
model = define_model(vocab_size, 30)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vgg19 (Model)                   multiple             20024384    input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
global_average_poolin

In [25]:
len(annot_train)

6068

In [26]:
#the below function loop forever with a while loop and within this, 
#loop over each image in the image directory. 
#For each image filename, we can load the image and 
#create all of the input-output sequence pairs from the image’s description.

#data generator, intended to be used in a call to model.fit_generator()
def data_generator(annots_train_r,img_train, max_length):
    while 1:
        for k in range(6068):
            #retrieve photo features
            img = img_train[k]
            annots_5 = annot_train[k]
            input_image, input_sequence, output_word = create_sequences( max_length, annots_5, img)
            yield [[input_image, input_sequence], output_word]

            
#we are calling the create_sequence() function to create 
#a batch worth of data for a single photo rather than an entire dataset. 
#This means that we must update the create_sequences() function 
#to delete the “iterate over all descriptions” for-loop.            
#Updated create sequence function for data_generator
def create_sequences(max_length, annots_5, img):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for annot in annots_5:
        # split one sequence into multiple X,y pairs
        for s in range(1, len(annot)):
            # split into input and output pair
            in_seq, out_seq = annot[:s], annot[s]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            X1.append(img)
            X2.append(in_seq)
            y.append(out_seq)
    
    return np.array(X1), np.array(X2), np.array(y)

In [27]:
# start_epoch = 0
# if ckpt_manager.latest_checkpoint:
#   start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

## 5 训练

In [ ]:
epochs = 20
steps = len(annot_train)
max_length = 30
# from time import time
# random.seed(time)
for i in range(epochs):
#    annot_train_r = []
#     for j in range(6068):
#         index = random.randint(0,4)
#         annot_train_r.append(annot_train[j][index])
    generator = data_generator(annot_train, img_train , max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    #model.fit(x={'inputs1': img_train, 'inputs2': annot_train_r}, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('model_' + str(i) + '.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
 711/6068 [==>...........................] - ETA: 55:45:37 - loss: 14.4534